# <b> Analysis Scripts for Montréal Urban Heat Waves </b>

The purpose of this file is to keep a record of scripts used to analyse CRCM6/GEM5 model output for the analysis of the urban heat island effect on historical heat waves. It includes the setup, analysis of observation and model output, and the code for plotting map data for the project.

# Table of Contents
1. <a href="#setup">Setup</a>
    - <a href="11-imports">1.1 Imports and Libraries</a>
    - <a href="#12-functions">1.2 Functions</a>
    - <a href="#13-datasets">1.2 Datasets</a>
2. <a href="#station-observations-and-model-output-data-visualisation">Station Observations and Model Output Data Visualisation</a>
    - <a href="#21-station-observations">2.1 Station Observations</a>
    - <a href="#22-comparison-of-stations-to-simulation-output">2.2 Comparison of Stations to Simulation Output</a>
    - <a href="#23-formatting-model-output">2.3 Formatting Model Output</a>

---

# 1. Setup
Common tools used throughout various scripts in the collection, analysis, and display of data throughout the project.

## <a id="11-imports"></a> 1.1 Imports

In [1]:
# !jupyter nbextension enable --py widgetsnbextension
import numpy as np 
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact

## <a id="12-functions"></a> 1.2 Functions
Custom statistical and numerical techniques usage and reference. 

In [2]:
#For projections of all map data used here
rotated_pole = ccrs.RotatedPole(pole_longitude=np.float32(106.425), pole_latitude=np.float32(44.5))

In [3]:
def seasonalMeans(dataset):
    """
    Calculates time average of a field for each season.

    Parameters
    ----------
    dataset : xarray.DataSet()
        The time series of the temperature of humidity field.

    Returns
    ----------
    xarray.DataSet()
        The seasonal means of the field.
    """

    return dataset.groupby('time.season').mean('time')

## <a id="13-datasets"></a> 1.3 Datasets

In [4]:
paths_ERA5_noTEB = glob('/home/documents/projects/runoff/gulley/St_Laurent/StLaurent_1km_SL2.5_ERA5_advHU/*tas.nc')
tas_ERA5_noTEB = xr.open_mfdataset(paths_ERA5_noTEB)

In [3]:
paths_ERA5_TEB = glob('/home/documents/projects/runoff/gulley/St_Laurent/StLaurent_1km_SL2.5_ERA5_advHU_TEB/*tas.nc')
tas_ERA5_TEB = xr.open_mfdataset(paths_ERA5_TEB)

In [5]:
tas_ERA5_noTEB

<xarray.Dataset> Size: 132GB
Dimensions:       (time: 210384, rlat: 280, rlon: 280, lev: 1)
Coordinates:
  * rlon          (rlon) float64 2kB -1.341 -1.332 -1.323 ... 1.152 1.161 1.17
  * rlat          (rlat) float64 2kB -1.251 -1.242 -1.233 ... 1.242 1.251 1.26
  * lev           (lev) float64 8B 1.5
  * time          (time) datetime64[ns] 2MB 1999-01-01T01:00:00 ... 2023-01-01
    lon           (rlat, rlon) float64 627kB dask.array<chunksize=(280, 280), meta=np.ndarray>
    lat           (rlat, rlon) float64 627kB dask.array<chunksize=(280, 280), meta=np.ndarray>
Data variables:
    rotated_pole  (time) |S1 210kB b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    tas           (time, rlat, rlon) float64 132GB dask.array<chunksize=(142, 122, 121), meta=np.ndarray>
Attributes: (12/26)
    Conventions:                    CF-1.4
    institution:                    Universite du Quebec a Montreal
    contact:                        Winger.Katja@uqam.ca
    comment:                        
    model:                          CRCM5 (dynamics GEM v_3.3.3, physics RPN ...
    model_grid:                     
    ...                             ...
    project_id:                     CORDEX
    CORDEX_domain:                  
    product:                        output
    references:                     http://www.mrcc.uqam.ca
    history:                        Thu Sep 26 14:18:30 2024: ncks -a -7 -L 1...
    NCO:                            netCDF Operators version 4.7.8 (Homepage ...

# 2. Station Observations and Model Output Data Visualisation
<!-- In this section:
- <a href="#21-station-observations">2.1 Station Observations</a>
- <a href="#22-comparison-of-stations-to-simulation-output">2.2 Comparison of Stations to Simulation Output</a>
- <a href="#23-formatting-model-output">2.3 Formatting Model Output</a> -->

## <a id="21-station-observations"></a>2.1 Station Observations
Multiyear monthly means to do mean annual cycles, annual means and seasonal means of temperature using station observations

## <a id="22-comparison-of-stations-to-simulation-output"></a>2.2 Comparison of Stations to Simulation Output
Select the closest grid cells of the simulations at 1km with and without TEB from few weather stations and compute multiyear monthly means to do mean annual cycles, annual means and seasonal means of temperature.

## <a id="23-formatting-model-output"></a>2.3 Formatting Model Output
Maps with python of seasonal means, and compute and plot differences between two simulations (with and without TEB)

In [6]:
ds = tas_ERA5_noTEB #contains the dataset to be displayed from the code for the interactive widget that follows

In [7]:
#For plotting and displaying model output contained in ds
def plot_temperature(time_index):
    """
    Plots the temperature field in the Montréal region of interest at a given time index of the ds xarray.Dataset()
    
    Parameters
    -----------
    time_index : int
        Index corresponding to some date within the dataset
    """
    
    plt.figure(figsize=(15, 9))
    data = ds['tas'].isel(time=time_index)
    ax = plt.subplot(1, 1, 1, projection=rotated_pole)
    ax.add_feature(cfeature.BORDERS,edgecolor='grey')
    ax.add_feature(cfeature.LAKES, edgecolor='grey', facecolor='none')
    ax.add_feature(cfeature.RIVERS, edgecolor='grey', facecolor='none')
    ax.add_feature(cfeature.COASTLINE,edgecolor='grey')
    mesh = ax.pcolormesh(ds['rlon'], ds['rlat'], data, transform=rotated_pole, cmap='coolwarm',vmin=223.15,vmax=323.15)
    plt.colorbar(mesh, orientation='vertical', label='Temperature (K)')
    plt.title(f'Temperature {ds.indexes['time'][time_index]}')
    plt.show()

# Play widget
play = widgets.Play(
    value=0,
    min=0,
    max=ds.sizes['time'] - 1,
    step=10,
    interval=100,  # Milliseconds between updates
    description="",
    disabled=False
)

# Slider linked to the play widget
slider = widgets.IntSlider(
    value=0,
    min=0,
    max=ds.sizes['time'] - 1,
    step=10,
    description = "",
    layout=widgets.Layout(width='1000px')
)

# Link the play widget and the slider
widgets.jslink((play, 'value'), (slider, 'value'))

#Displaying the map with slider and play widget
display(play)
time_slider = interact(
    plot_temperature,
    time_index=slider,
)


Play(value=0, max=210383, step=10)

interactive(children=(IntSlider(value=0, description='time_index', layout=Layout(width='1000px'), max=210383, …

In [ ]:
seasonal_means = seasonalMeans(ds)

seasons = ['DJF', 'MAM', 'JJA', 'SON']  #December January Feb, March April May, June July August, September October November
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for ax, season in zip(axes.flat, seasons):
    data = seasonal_means['tas'].sel(season=season)
    mesh = ax.pcolormesh(seasonal_means['rlon'], seasonal_means['rlat'], data, transform=rotated_pole, cmap='coolwarm',vmin=223.15,vmax=323.15)
    plt.colorbar(mesh, orientation='vertical', label='Temperature (K)')
    ax.set_title(f'{season} Average Temperature')

plt.tight_layout()
plt.show()

Todo: 
 - How the urban effect can be seen through climatology 
 - Impact of TEB at particular points such as station in and outside city centre, compute diffs from station data w/wo TEB
 - Do that for multiple stations and observe the effects
 - seasonal averages
 - Verify against station data
 - Investigate urban fraction field
 - Max and mim fields

Over the next two weeks
   Spatial seasonally averaged maps of T, TMAX, and TMIN
   Check out figure 4 and 16 of the report
   
Definition of the UHI